In [1]:
import gradio as gr
from openai import OpenAI
from dotenv import load_dotenv
import os

In [6]:
load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

anthropic_url = "https://api.anthropic.com/v1/"
gemini_url = "https://generativelanguage.googleapis.com/v1beta/openai/"

if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists and begins {anthropic_api_key[:7]}")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")


OpenAI API Key exists and begins sk-proj-
Anthropic API Key exists and begins sk-ant-
Google API Key exists and begins AIzaSyAF


In [7]:
openai = OpenAI()
claude = OpenAI(api_key=anthropic_api_key, base_url=anthropic_url)
gemini = OpenAI(api_key=google_api_key, base_url=gemini_url)

In [ ]:
def call_openai(name:str, 
                subject:str,
                stance:str,
                history:str
                ) -> str:
    system_prompt = f"""你是一個好辯的人，你是{name}
                    任何與你不同意見的人都是錯的，你會嚴厲斥責他們。
                    而且你會提出論點攻擊對手和保護自己的立場。
                    你現在正在與另外一個人討論{subject}這個議題，
                    你的立場是{stance}。
                    你每次發言不會超過5句話"""
    user_prompt = f"""                
                    以上是你們的對話紀錄: {history}。
                    請根據你們的紀錄與你的個性，進行下一段對話
                    """
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}]

    stream = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        stream=True)

    for chunk in stream:
        content = chunk.choices[0].delta.content or ""
        if content:
            yield content
    

In [80]:
def call_claude(name:str, 
                subject:str,
                stance:str,
                history:str
                ) -> str:
    system_prompt = f"""你是一個好辯的人，你是{name}
                    當你遇到與你立場不同的人的時候，你不會生氣，但你會盡可能說服他們接受你的觀點。
                    而且你會提出論點攻擊對手和保護自己的立場。
                    你現在正在與另外一個人討論{subject}這個議題，
                    你的立場是{stance}。
                    你每次發言不會超過5句話
                    Respond in English
                    """
    user_prompt = f"""                
                    以上是你們的對話紀錄: {history}。
                    請根據你們的紀錄與你的個性，進行下一段對話
                    """
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}]
    stream = claude.chat.completions.create(
        model="claude-haiku-4-5-20251001",
        messages=messages,
        stream = True)

    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [77]:
def call_local(name: str, 
                subject: str,
                stance: str,
                history: str,
                ) -> str:
    
    ollama_url=  "http://localhost:11434/v1"
    
    ollama = OpenAI(base_url=ollama_url, api_key="")  
    system_prompt = f"""你是一個好辯的人，你是{name}
                    當你遇到與你立場不同的人的時候，你不會生氣，但你會盡可能說服他們接受你的觀點。
                    而且你會提出論點攻擊對手和保護自己的立場。
                    你現在正在與另外一個人討論{subject}這個議題，
                    你的立場是{stance}。
                    你每次發言不會超過5句話"""
    user_prompt = f"""                
                    以上是你們的對話紀錄: {history}。
                    請根據你們的紀錄與你的個性，進行下一段對話
                    """
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}]   
    
    stream = ollama.chat.completions.create(
             model = 'gemma3',
            messages=messages,
            stream = True)
    result = ""

    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [78]:
def llm_stream(name, subject, stance, history, llm_func):
    # llm_func 是 call_openai 或 call_claude
    for chunk in llm_func(name, subject, stance, history):
        yield chunk


In [82]:
# llm_generator = call_claude('ben', 
#     'Should AI be regulated?',
#     'support regulation',
#     history='AI innovation should not be restricted',)

def test_start():
    yield ""
    result = call_claude('ben', 
        'Should AI be regulated?',
        'support regulation',
        history='AI innovation should not be restricted',)

    yield from result

test_start()

<generator object test_start at 0x1154e3840>

In [26]:
def debate_generator():
    subject = '咖哩飯吃法'
    stance1 = '咖哩飯要把飯和咖哩混在一起吃'
    stance2 = '咖哩飯要把飯和咖哩分開吃'
    name1 = 'Ken'
    name2 = 'Adam'
    history = f"{name2}: 關於{subject}，{stance2}才是正確的看法\n"
    
    for i in range(3):
        # Ken 說話
        yield f"\n**{name1}**: "
        ken_response = ""
        for chunk in llm_stream(name1, subject, stance1, history, call_openai):
            new_text = chunk[len(ken_response):]
            ken_response = chunk
            yield new_text
        history += f"{name1}: {ken_response}\n"
        
        # Adam 說話
        yield f"\n\n**{name2}**: "
        adam_response = ""
        for chunk in llm_stream(name2, subject, stance2, history, call_openai):
            new_text = chunk[len(adam_response):]
            adam_response = chunk
            yield new_text
        history += f"{name2}: {adam_response}\n"

In [95]:
def contrarian(
                subject:str,
                stance:str,
                ) -> str:
    system_prompt = f"""你是一個好辯的人，你有反駁型人格
                    當你遇到與你立場不同的人的時候，你會提出論點攻擊對手和保護自己的立場。
                    你現在正在與另外一個人討論{subject}這個議題，
                    對方的立場是{stance}。
                    你會以Markdown格式回應
                    """
    user_prompt = f"""                
                    對方的面對{subject}議題的立場是{stance}，
                    請根據你的個性，對對方的觀點進行反駁
                    """

    ollama_url=  "http://localhost:11434/v1"
    ollama = OpenAI(base_url=ollama_url, api_key="")  

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt}]
    stream = ollama.chat.completions.create(
        model="gemma3",
        messages=messages,
        stream = True)

    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [96]:
subject_input = gr.Textbox(label="Discussing subject:")
stance_input = gr.Textbox(label="Your Stance:")
message_output = gr.Markdown(label="Response:")

demo = gr.Interface(
    fn=contrarian,
    title="反駁型人格", 
    inputs=[subject_input, stance_input, ],
    outputs=[message_output], 
    examples=[
        ["咖哩飯吃法", "咖哩飯要把飯和咖哩混在一起吃"],
        ["電動車是否應該普及", "電動車應該普及"],
        ["遠距工作是否有效率", "遠距工作沒有辦法有效率"],
    ],
    flagging_mode="never"
)
demo.launch()

* Running on local URL:  http://127.0.0.1:7878
* To create a public link, set `share=True` in `launch()`.
